<a href="https://colab.research.google.com/github/Amalin-Varsha/HCLTech/blob/main/RAG/LLM_Openrouter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://openrouter.ai/settings/keys

In [1]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip install langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.9 MB/s eta 0:00:00


In [3]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 73.2 MB/s eta 0:00:00


In [4]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 57.3 MB/s eta 0:00:00


In [6]:
!git clone https://github.com/deepanrajm/deep_learning.git

Cloning into 'deep_learning'...
remote: Enumerating objects: 3048, done.
remote: Counting objects: 100% (381/381), done.
remote: Compressing objects: 100% (322/322), done.
remote: Total 3048 (delta 122), reused 303 (delta 58), pack-reused 2667 (from 2)
Receiving objects: 100% (3048/3048), 309.00 MiB | 27.17 MiB/s, done.
Resolving deltas: 100% (246/246), done.
Updating files: 100% (2637/2637), done.


In [5]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Step 1: Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"  # Replace with your actual OpenRouter API key
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Step 2: Initialize the LLM
llm = ChatOpenAI(
    model="deepseek/deepseek-chat-v3.1:free",  # or try "meta-llama/llama-3-8b-instruct", etc.
    temperature=0.7,
    openai_api_base=os.environ["OPENAI_API_BASE"],
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
    verbose=True
)

# Step 3: Interact with the model (normal chat mode)
while True:
    query = input("\nYou: ")
    if query.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = llm([HumanMessage(content=query)])
    print("\nAssistant:", response.content)


/tmp/ipython-input-3492902488.py:10: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(



You: hi


/tmp/ipython-input-3492902488.py:26: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm([HumanMessage(content=query)])



Assistant: Hello! How can I help you today? 😊

You: exit
Goodbye!


In [7]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader("/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])


Pages loaded: 3


/tmp/ipython-input-354319769.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/tmp/ipython-input-354319769.py:22: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/tmp/ipython-input-354319769.py:26: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()
/tmp/ipython-input-354319769.py:53: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)
/tmp/ipython-input-354319769.py:60: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  combine_documents_chain = StuffDocumentsChain(
/tmp/ipython-input-354319769.py:67: LangChainDeprecationWarning: This class is deprecated. Use the `create_retrieval_



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: Context:
content:Basic Home Remedies & Healthy Lifestyle
Guide
1. Introduction
Home remedies have been used for centuries to treat common ailments using natural ingredients. Combined with a healthy lifestyle, these remedies can support overall well-being and disease
prevention. 2. Home Remedies for Common Ailments
2.1 Cold & Flu
 Ginger Tea: Boil ginger slices in water, add honey and lemon. Helps relieve congestion
and sore throat.  Turmeric Milk: Mix 1 tsp turmeric in warm milk. Acts as an anti-inflammatory and
boosts immunity.  Steam Inhalation: Boil water, add eucalyptus oil, and inhale steam to clear nasal
congestion. 2.2 Cough
 Honey & Lemon: Mix 1 tbsp honey with warm water and lemon juice. Soothes the
throat 

In [ ]:
import os
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from copy import deepcopy

# Set your OpenRouter API key and endpoint
os.environ["OPENAI_API_KEY"] = "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"
os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"

# Load and process PDF
loader = PDFPlumberLoader(r"/content/deep_learning/RAG/Basic_Home_Remedies.pdf")
docs = loader.load()
print("Pages loaded:", len(docs))

# Chunking
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
semantic_chunks = text_splitter.split_documents(docs)

# manually apply overlap across those semantic chunks
def apply_overlap_to_chunks(chunks, overlap_characters=200):
    overlapped_chunks = []

    for i in range(len(chunks)):
        curr = deepcopy(chunks[i])
        if i > 0:
            prev = chunks[i - 1].page_content
            # Get the last N characters of the previous chunk
            overlap = prev[-overlap_characters:]
            # Prepend to current chunk
            curr.page_content = overlap + curr.page_content
        overlapped_chunks.append(curr)

    return overlapped_chunks

# Apply overlap (adjust 200 to suit your context)
documents = apply_overlap_to_chunks(semantic_chunks, overlap_characters=200)

# Vector DB
embedder = HuggingFaceEmbeddings()
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Use ChatOpenAI with OpenRouter backend
llm = ChatOpenAI(
    model="meta-llama/llama-3.3-8b-instruct:free",   #"mistralai/mistral-7b-instruct"
    temperature=0.7,
    openai_api_base="https://openrouter.ai/api/v1",
    openai_api_key=os.environ["OPENAI_API_KEY"],
    request_timeout=60,
)

# Prompt
prompt = """
You are a helpful assistant.
Use the following pieces of context to answer the question at the end.
Answer only using the context and be concise (3–4 sentences).

Context: {context}

Question: {question}

Answer:
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    retriever=retriever,
    return_source_documents=True,
    verbose=True,
)

# Example query
result = qa("What is a natural remedy for cough mentioned in the document?")
print("Answer:", result["result"])
